In [61]:
import numpy as np
import pandas as pd
import os

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('../input/ml100k/ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

m_cols = ['movie_id', 'title']
movies = pd.read_csv('../input/ml100k/ml-100k/u.item', sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")

ratings = pd.merge(movies, ratings) # Merge both movie and ratings to get a dataframe which has user, rating, and movie.

ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [62]:
movieRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating') # Create matrix of user and the movies that user have rated

movieRatings.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN


In [64]:
movieChosen = input("Input Name_of_movie (Year_released)") # Raiders of the Lost Ark (1981) is written in output

chosenFilmRatings = movieRatings[movieChosen] # Assign values of all user chosen film to a dataframe

chosenFilmRatings.head()

Input Name_of_movie (Year_released)Raiders of the Lost Ark (1981)


user_id
1    5.0
2    NaN
3    NaN
4    NaN
5    5.0
Name: Raiders of the Lost Ark (1981), dtype: float64

In [69]:
similarMovies = movieRatings.corrwith(chosenFilmRatings) # Pairing corrwith of film vector of user rating with movies

similarMovies = similarMovies.dropna() # Remove NaN ratings

df = pd.DataFrame(similarMovies) # Make a Dataframe of movies and their pairing score to the chosen film

df.head(10)

/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,0
title,
'Til There Was You (1997),0.975900
1-900 (1994),-0.944911
101 Dalmatians (1996),0.091254
12 Angry Men (1957),0.129334
187 (1997),0.230769
2 Days in the Valley (1996),0.031321
"20,000 Leagues Under the Sea (1954)",0.398606
2001: A Space Odyssey (1968),0.198027
"39 Steps, The (1935)",0.230426


In [70]:
similarMovies.sort_values(ascending=False) # Sort by score

movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})

popularMovies = movieStats['rating']['size'] >= 100 # Ignore movies rated by less than 100 people to get more sensible results

movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                         size      mean
title                                                  
Close Shave, A (1995)                     112  4.491071
Schindler's List (1993)                   298  4.466443
Wrong Trousers, The (1993)                118  4.466102
Casablanca (1942)                         243  4.456790
Shawshank Redemption, The (1994)          283  4.445230
Rear Window (1954)                        209  4.387560
Usual Suspects, The (1995)                267  4.385768
Star Wars (1977)                          583  4.358491
12 Angry Men (1957)                       125  4.344000
Citizen Kane (1941)                       198  4.292929
To Kill a Mockingbird (1962)              219  4.292237
One Flew Over the Cuckoo's Nest (1975)    264  4.291667
Silence of the Lambs, The (1991)          390  4.289744
North by Northwest (1959)                 179  4.284916
Godfather, The (1972)                     413  4.283293

In [60]:
df = movieStats[popularMovies].join(pd.DataFrame(similarMovies, columns=['similarity']))

df.sort_values(['similarity'], ascending=False)[:5] # Sort by similarity score

,"(rating, size)","(rating, mean)",similarity
title,,,
Star Wars (1977),583,4.358491,1.000000
"Empire Strikes Back, The (1980)",367,4.204360,0.747981
Return of the Jedi (1983),507,4.007890,0.672556
Raiders of the Lost Ark (1981),420,4.252381,0.536117
Austin Powers: International Man of Mystery (1997),130,3.246154,0.377433
